# OpenAI API 사전 준비

In [ ]:
# 패키지 설치
!pip install -U openai

In [ ]:
import os
from google.colab import userdata

# 환경 변수 준비 (왼쪽 끝 키 아이콘으로 OPENAI_API_KEY 설정)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

# 클라이언트 준비
client = OpenAI()

# 임베딩 실행

In [ ]:
# 텍스트 준비
text = "이것은 테스트입니다."

In [ ]:
# 텍스트를 벡터 표현으로 변환
response =client.embeddings.create(
    input=text,
    model="text-embedding-3-small"
)
print(response.data[0].embedding)

In [ ]:
# 벡터 표현의 길이 확인
print(len(response.data[0].embedding))

# 근접 탐색

In [ ]:
# Faiss 패키지 설치
!pip install faiss-cpu

In [ ]:
import numpy as np

# 입력 텍스트를 벡터 표현으로 변환
in_text = "오늘은 비가 안 와서 다행이야"
response =client.embeddings.create(
    input=in_text,
    model="text-embedding-3-small"
)

# numpy 배열로 변환
in_embeds = [record.embedding for record in response.data]
in_embeds = np.array(in_embeds).astype("float32")

In [ ]:
# 대상 텍스트를 벡터 표현으로 변환
target_texts = [
    "좋아하는 음식은 무엇인가요?",
    "어디에 사시나요?",
    "아침 전철은 붐비네요",
    "오늘은 날씨가 좋네요",
    "요즘 경기가 나쁘네요"]
response =client.embeddings.create(
    input=target_texts,
    model="text-embedding-3-small"
)

# numpy 배열로 변환
target_embeds = [record.embedding for record in response.data]
target_embeds = np.array(target_embeds).astype("float32")

In [ ]:
import faiss

# Faiss 인덱스 생성
index = faiss.IndexFlatL2(1536)

In [ ]:
# 대상 텍스트를 인덱스에 추가
index.add(target_embeds)

In [ ]:
# 입력 텍스트로 근접 탐색 실행
distances, indices = index.search(in_embeds, 1)

# 확인
print("distance:", distances[0][0])
print("index:", indices[0][0])
print("target_text:", target_texts[indices[0][0]])